In [1]:
%matplotlib inline
from __future__ import division, print_function
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import sys, os
sys.path.append('/Users/roz18/git/Python/user_modules')
from readEazyBinary import readpz
import scipy.interpolate
import scipy.integrate
import scipy

In [2]:
path = '/Users/roz18/Dropbox/Work/mini-projects/Buzzard-EAZY/inputs_Buzzard_training/OUTPUT'

cat = Table.read('/Users/roz18/Documents/Data/mini-projects/buzzard_photo-z/Final_Buzzard_training_file.fits')
print(len(cat))

# Gold sample cut
mask = cat['LSSTi']<25.3
print(np.sum(mask))

# Remove objects with invalide flux measurements
mask = mask & (cat['LSSTu']!=99.) & (cat['LSSTg']!=99.) & (cat['LSSTr']!=99.) & \
       (cat['LSSTi']!=99.) & (cat['LSSTz']!=99.) & (cat['LSSTy']!=99.)
print(np.sum(mask))

# redshift range
zmin = 0.
zmax = 2.0
mask = mask & (cat['redshift']>zmin) & (cat['redshift']<zmax)
print(np.sum(mask))
cat = cat[mask]
zspec = cat['redshift']

111171
44410
43486
43486


In [3]:
# Load p(z) data
pz_dict = readpz(MAIN_OUTPUT_FILE='photz', OUTPUT_DIRECTORY=path, APPLY_PRIOR=False)
# Redshift grid
zgrid = pz_dict['zgrid']
# p(z)
pz = pz_dict['pz'][mask]
# prior = pz_dict['prior'][mask]
# chi squared
chi2 = pz_dict['chi2'][mask]

In [4]:
pz_dict

{'chi2': array([[  2.08913096e+01,   2.15143985e+01,   3.87740807e+01, ...,
           6.57633592e+03,   6.25808268e+03,   5.97557358e+03],
        [  3.69578669e+01,   4.03184124e+01,   3.14300230e+01, ...,
           1.89012388e+03,   1.74546979e+03,   1.62759783e+03],
        [  2.66153333e+02,   2.41853984e+02,   2.20307112e+02, ...,
           4.88446925e+03,   4.56660893e+03,   4.29711428e+03],
        ..., 
        [  4.62318850e+00,   4.96292984e+00,   5.58831447e+00, ...,
           3.15177836e+00,   3.24934391e+00,   3.37153269e+00],
        [  1.14301998e+01,   1.14369697e+01,   1.15278571e+01, ...,
           6.49970954e+00,   6.79035055e+00,   7.11511942e+00],
        [  1.71820357e+01,   1.67575986e+01,   1.63714776e+01, ...,
           2.67437146e+01,   2.77227251e+01,   2.86765364e+01]]),
 'pz': array([[  2.00084303e+00,   1.46524746e+00,   2.61829518e-04, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  2.61905613e-03,   4.87990792e-04,  

In [5]:
# Downsampling
size = 100
np.random.seed(1842)
idx = np.random.choice(len(pz), size=size, replace=False)
pz = pz[idx]
zspec = zspec[idx]

In [6]:
np.savetxt('/Users/roz18/Documents/git/PZDC1paper/mystuff/pz.txt', pz)
np.savetxt('/Users/roz18/Documents/git/PZDC1paper/mystuff/zgrid.txt', zgrid)
np.savetxt('/Users/roz18/Documents/git/PZDC1paper/mystuff/zspec.txt', zspec)

__Interpolate p(z):__

In [7]:
# Insert points between the grid points
# set the number of iterations
n_iterations = 3
zgrid_fine = zgrid.copy()
for index in range(n_iterations):
    zgrid1 = np.zeros(len(zgrid_fine)*2-1)
    zgrid1[::2] = zgrid_fine
    zgrid1[1::2] = (zgrid_fine[:-1]+zgrid_fine[1:])/2
    zgrid_fine = zgrid1.copy()
    
pz_fine = np.zeros((len(pz), len(zgrid_fine)))
for index in range(len(pz)):
    pz_interp = scipy.interpolate.interp1d(zgrid, pz[index], kind='linear')
    pz_fine[index] = pz_interp(zgrid_fine)
    print('\r{}'.format(index), end='')

99

In [8]:
np.savetxt('/Users/roz18/Documents/git/PZDC1paper/mystuff/zgrid_fine.txt', zgrid_fine)
np.savetxt('/Users/roz18/Documents/git/PZDC1paper/mystuff/pz_fine.txt', pz_fine)